# bind runtime args

Sometimes we want to invoke a Runnable within a Runnable sequence with constant arguments that are not part of the output of the preceding Runnable in the sequence, and which are not part of the user input. We can use Runnable.bind() to easily pass these arguments in.

Suppose we have a simple prompt + model sequence:

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, module_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)

runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12\nSOLUTION: x^3 = 12 - 7\nx^3 = 5\nx =立方根(5)\nx ≈ 1.442289524\n\n因此，方程的解为 x ≈ 1.442289524。


and want to call the model with certain stop words:

In [4]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x³ + 7 = 12\nSOLUTION:\n\nTo solve this equation, we'll first subtract 7 from both sides:\n\nx³ = 12 - 7\nx³ = 5\n\nNow, we need to find the value of x that will raise to the third power to equal 5. Since we're looking for a cube root, we can use the symbol ∛ to represent the cube root of a number. So, the solution is:\n\nx = ∛5


## Attaching OpenAI functions
One particularly useful application of binding is to attach OpenAI functions to a compatible OpenAI model:

In [5]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [6]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = model.bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

'Algebraic equation: x^3 + 7 = 12\\n\\nTo solve this equation, we can subtract 7 from both sides:\\n\\nx^3 = 12 - 7\\n\\nx^3 = 5\\n\\nNow, we can solve for x by taking the立方根 of both sides:\\n\\nx = 立方根(5)\\n\\nSo, the solution is x = 立方根(5).'

Attaching OpenAI tools

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [8]:
model = model.bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

'As an AI language model, I cannot provide real-time weather updates. However, I can guide you on how to check the weather in these cities:\\n\\n1. San Francisco (SF):\\nGo to the weather forecast website like AccuWeather or Weather.com and search for \\"San Francisco\\" or the airport code \\"SFO\\" to get the current weather and forecasts.\\n2. New York City (NYC):\\n同样，访问天气预报网站，如AccuWeather或Weather.com，搜索\\"New York\\"或机场代码\\"JFK\\"来获取纽约市的当前天气和预报。\\n3. Los Angeles (LA):\\n同样，访问天气预报网站，如AccuWeather或Weather.com，搜索\\"Los Angeles\\"或机场代码\\"LAX\\"来获取洛杉矶的当前天气和预报。\\n\\nThese websites provide weather updates, forecasts, and radar maps for these cities. Remember that weather can change rapidly, so it\'s best to check the latest updates.'